In [151]:
import pandas as pd
import numpy as np
import os

In [152]:
train_files = [i for i in os.listdir('results/') if '_val_reg' in i]
test_files = [i for i in os.listdir('results/') if '_test_reg' in i]

train_files.remove('y_val_reg')
train_files = sorted(train_files)
test_files = sorted(test_files)

In [153]:
y_train = np.fromfile("results/y_val_reg", dtype='float32')

In [154]:
train = pd.DataFrame({"var" + str(i) : np.fromfile("results/" + file, dtype='float32') for i, file in enumerate(train_files)})
y = np.fromfile("results/y_val_reg", dtype='float32')

test = pd.DataFrame({"var" + str(i) : np.fromfile("results/" + file, dtype='float32') for i, file in enumerate(test_files)})


In [155]:
y_final_preds = np.round(np.clip(test, 1, 10).mean(axis=1)).astype(int)

In [156]:
TestX = pd.read_csv("data/TestX.csv", sep=";")
Train = pd.read_csv("data/Train.csv", sep=";")

Train = Train.drop_duplicates(['condition', 'opinion']).reset_index(drop=True)

submission = pd.read_csv("data/JANLOS.txt", sep=" ")

In [157]:
TestX_merged = TestX.merge(Train, on=['opinion', 'condition'], how='left')

In [159]:
TestX_merged['y_final_preds'] = y_final_preds

In [160]:
TestX_merged.head()

,name_x,condition,opinion,name_y,rate,rate1,y_final_preds
0,Vitamin B12,Vitamin B12 Deficiency,"""No after taste at unlike all the others that ...",Cyanocobalamin,10.0,high,8
1,Implanon,Birth Control,"""I have not had any issues with the implanon I...",Etonogestrel,8.0,high,8
2,BuSpar,Anxiety,"""I have been taking Buspar and Celexa and as f...",Buspirone,10.0,high,9
3,Abreva,Herpes Simplex,"""the only thing that works. completely awesome.""",Docosanol,10.0,high,10
4,Buprenorphine / naloxone,Opiate Dependence,"""UPDATE: I originally posted on Nov 16th 2010 ...",Suboxone,10.0,high,10


In [161]:
TestX_merged["rate_final"] = TestX_merged['rate'].fillna(TestX_merged['y_final_preds'])
TestX_merged.rate_final = TestX_merged.rate_final.astype(int)

In [162]:
TestX_merged.rate_final.value_counts()

10    13430
9     11405
8      5622
1      5134
7      2995
2      2788
3      2337
5      2292
6      2090
4      1907
Name: rate_final, dtype: int64